In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import matplotlib as mpl
import cv2
from matplotlib import patches
from time import time

In [2]:
def OnMouseAction(event, x, y, flags, param):
    global img, position1, position2, flag
    if event == cv2.EVENT_LBUTTONDOWN:  # Press the left mouse button
        position1 = (x, y)
        position2 = None

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:  # Hold down the left mouse button and drag
        position2 = (x, y)

    elif event == cv2.EVENT_LBUTTONUP:  # Release the left mouse button
        position2 = (x, y)
        flag = True

In [3]:
def plot_tracking(image, tlwhs, scores=None, frame_id=0, fps=0., ids2=None):
    im = np.ascontiguousarray(np.copy(image))
    im_h, im_w = im.shape[:2]

    top_view = np.zeros([im_w, im_w, 3], dtype=np.uint8) + 255

    text_scale = max(1, image.shape[1] / 1600.)
    text_thickness = 2
    line_thickness = max(1, int(image.shape[1] / 500.))

    radius = max(5, int(im_w / 140.))
    cv2.putText(im, 'frame: %d fps: %.2f num: %d' % (frame_id, fps, len(tlwhs)),
                (0, int(15 * text_scale)), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255), thickness=1)

    for i, tlwh in enumerate(tlwhs):
        obj_ids, x1, y1, w, h = tlwh
        intbox = tuple(map(int, (x1, y1, x1 + w, y1 + h)))
        obj_id = int(obj_ids)
        id_text = '{}'.format(int(obj_id))
        if ids2 is not None:
            id_text = id_text + ', {}'.format(int(ids2[i]))
        _line_thickness = 1 if obj_id <= 0 else line_thickness
        color = get_color(abs(obj_id))
        cv2.rectangle(im, intbox[0:2], intbox[2:4], color=color, thickness=line_thickness)
        cv2.putText(im, id_text, (intbox[0], intbox[1] + 30), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255),
                    thickness=text_thickness)
    return im

In [4]:
def get_color(idx):
    idx = idx * 3
    color = ((37 * idx) % 255, (17 * idx) % 255, (29 * idx) % 255)
    return color

train_path = Path("/Users/86136/Desktop/M.YY.L/9517/9517collect/data/step_images/train/")
label_path = Path("/Users/86136/Desktop/M.YY.L/9517/9517collect/data/panoptic_maps/train/")
test_path = Path("/Users/86136/Desktop/M.YY.L/9517/9517collect/data/step_images/test/")
train_id = ["0002", "0009"]
train_frame = [600, 525]
h, w = 1080, 1920

output_path = "/Users/86136/Desktop/M.YY.L/9517/9517collect/output/"
test_name = "STEP-ICCV21-01"
test_frame = {"STEP-ICCV21-01": 450, "STEP-ICCV21-07": 500}

position1 = None
position2 = None
flag = False

label = str(Path(output_path) / test_name / "results.txt")
tracks = []
track_frame = []
pre_frame = 0

with open(label) as f:
    for line in f:
        stripe = line.split(",")
        frame = int(stripe[0])
        records = [float(i) for i in stripe[1:6]]
        records[0] = int(records[0])
        if frame != pre_frame:
            tracks.append(track_frame)
            track_frame = [records]
            pre_frame = frame
        else:
            track_frame.append(records)
del tracks[0]

cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
cv2.resizeWindow("frame", 1280, 720)
cv2.setMouseCallback('frame', OnMouseAction)
img = cv2.imread(str(test_path / test_name / "000001.jpg"))
cv2.imshow('frame', img)
cv2.waitKey(1)

img = None
trajectory = {}

while 1:
    if flag:
        for idx in range(1, test_frame[test_name] + 1):
            print(idx)
            # draw box first
            test_file = test_path / test_name / ('%06d' % idx + ".jpg")
            img = cv2.imread(str(test_file))

            # box
            img = plot_tracking(img, tracks[idx], frame_id=idx, fps=0.)

            # trajectory
            tracking = tracks[idx]
            for t in tracking:
                _id = t[0]
                if _id in trajectory:
                    trajectory[_id].append((int(t[1] + t[3] / 2), int(t[2] + t[4] / 2)))
                else:
                    trajectory[_id] = [(int(t[1] + t[3] / 2), int(t[2] + t[4] / 2))]
            for _id, center in trajectory.items():
                for i in range(len(center) - 1):
                    cv2.line(img, center[i], center[i + 1], thickness=1, color=get_color(_id))

            # roi
            cv2.rectangle(img, position1, position2, (0, 255, 0), 2)
            cnt = 0
            for t in tracking:
                x1, y1, w, h = t[1:]
                if x1 > position1[0] and y1 > position1[1] and x1 + w < position2[0] and y1 + h < position2[1]:
                    cnt += 1
            text_scale = max(1, img.shape[1] / 1600.)
            cv2.putText(img, 'roi num: %d' % cnt,
                        (position1[0], position1[1] - 5),
                        cv2.FONT_HERSHEY_PLAIN,
                        text_scale, (0, 255, 0), thickness=1)

            # img.resize(1280, 720)
            cv2.imshow('frame', img)
            cv2.waitKey(1)
    cv2.waitKey(1)

KeyboardInterrupt: 